In [1]:
import math
from pathlib import Path
import sys
import numpy as np
import pandas as pd
from scipy import stats
import statistics

from sklearn.model_selection import train_test_split

In [2]:
import tcremb.data_proc as data_proc

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
outputs_path = 'data/data_preped/'

In [5]:
#tcr_columns = {'single': ['cdr3aa','v','j','chain'],'paired': {'TRA': ['a_cdr3aa', 'TRAV', 'TRAJ'],'TRB': [ 'b_cdr3aa', 'TRBV', 'TRBJ']}}
tcr_columns = ['cdr3aa','v','j','chain']
clonotype_id_column = 'cloneId'
data_id= 'data_id'
annotation_id = 'annotId'
pairing_id = 'barcode'
annotation_tcr_id_columns_dict = {'TRA': 'cloneId','TRB': 'cloneId','TRA_TRB': {'TRA':'cloneId_TRA', 'TRB':'cloneId_TRB'}}

## Prototypes

### TRA

In [11]:
data_raw_path = 'data/olga_humanTRA_3000.txt'
data_output_path = outputs_path +'olga_humanTRA.txt'

In [12]:
data = pd.read_csv(data_raw_path,sep='\t',header=None)
data = data.rename({0:'cdr3nt',1:'cdr3aa',2:'v',3:'j'},axis=1)

In [13]:
data_preped = data_proc.remove_asterisk(data, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.remove_backslash(data_preped, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.filter_clones_data(data_preped, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.filter_segments(data_preped)
data_preped.shape

(3000, 4)
(3000, 4)


(2951, 4)

In [14]:
data_preped.to_csv(data_output_path, sep='\t',header=False)

### TRB

In [15]:
data_raw_path = 'data/olga_humanTRB_3000.txt'
data_output_path = outputs_path +'olga_humanTRB.txt'

In [16]:
data = pd.read_csv(data_raw_path,sep='\t',header=None)
data = data.rename({0:'cdr3nt',1:'cdr3aa',2:'v',3:'j'},axis=1)

In [17]:
data_preped = data_proc.remove_asterisk(data, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.remove_backslash(data_preped, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.filter_clones_data(data_preped, ['cdr3nt','cdr3aa','v','j'])
data_preped = data_proc.filter_segments(data_preped)
data_preped.shape

(3000, 4)
(3000, 4)


(3000, 4)

In [18]:
data_preped.to_csv(data_output_path, sep='\t',header=False)

## VDJdb

In [6]:
label = 'antigen.epitope'
label_s = 'antigen.species'

In [7]:
data_raw_path = 'data/vdjdb.slim.txt'
data_tra_output_path = outputs_path +'VDJdb_data_TRA.csv'
data_trb_output_path = outputs_path +'VDJdb_data_TRB.csv'

data_output_path = outputs_path +'VDJdb_data.csv'
data_train_output_path = outputs_path +'VDJdb_data_train.csv'
data_test_output_path = outputs_path +'VDJdb_data_test.csv'

In [8]:
data = pd.read_csv(data_raw_path,sep='\t')
data = data.rename({'cdr3':'cdr3aa','gene':'chain','v.segm':'v','j.segm':'j'},axis=1)




In [9]:
data

,chain,cdr3aa,species,antigen.epitope,antigen.gene,antigen.species,complex.id,v,j,v.end,j.start,mhc.a,mhc.b,mhc.class,reference.id,vdjdb.score
0,TRB,CASSTSRLSNQPQYF,MacacaMulatta,STPESANL,Tat,SIV,0,TRBV14*01,TRBJ1-5*01,4,8,Mamu-A*01,B2M,MHCI,PMID:19349463,1
1,TRB,CASSQDRGPANEQFF,HomoSapiens,RLRAEAQVK,EBNA3A,EBV,21729,TRBV4-1*01,TRBJ2-1*01,5,10,HLA-A*03:01,B2M,MHCI,https://www.10xgenomics.com/resources/applicat...,0
2,TRB,CASSMSRSSNQPQYF,MacacaMulatta,TTPESANL,Tat,SIV,0,TRBV14*01,TRBJ1-5*01,4,8,Mamu-A*01,B2M,MHCI,PMID:15589168,0
3,TRA,CASNTGTASKLTF,HomoSapiens,GILGFVFTL,M,InfluenzaA,0,TRAV24*01,TRAJ44*01,2,3,HLA-A*02,B2M,MHCI,PMID:28423320,0
4,TRB,CASSLGSQNTLYF,MusMusculus,HGIRNASFI,M45,MCMV,"24797,24785,24840,24778,24833",TRBV12-1*01,TRBJ2-4*01,5,6,H-2Db,B2M,MHCI,PMID:28636592,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75275,TRB,CASSWGLYNEQFF,HomoSapiens,MEVTPSGTWL,Nucleocapsid,SARS-CoV-2,0,TRBV27*01,TRBJ2-1*01,4,7,HLA-B*40:01,B2M,MHCI,PMID:35750048,0
75276,TRA,CALLNRDDKIIF,HomoSapiens,RLRAEAQVK,EBNA3A,EBV,10402,TRAV9-2*01,TRAJ30*01,3,4,HLA-A*03:01,B2M,MHCI,https://www.10xgenomics.com/resources/applicat...,0
75277,TRB,CASSYSQETQYF,HomoSapiens,KLGGALQAK,IE1,CMV,14228,TRBV6-3*01,TRBJ2-5*01,5,6,HLA-A*03:01,B2M,MHCI,https://www.10xgenomics.com/resources/applicat...,0
75278,TRB,CAWTSGLGNEQFF,HomoSapiens,KLGGALQAK,IE1,CMV,16987,TRBV30*01,TRBJ2-1*01,3,8,HLA-A*03:01,B2M,MHCI,https://www.10xgenomics.com/resources/applicat...,0


### Assign data id

In [10]:
data = data_proc.annot_id(data, data_id)

### data proc

In [11]:
data_preped = data_proc.remove_asterisk(data, tcr_columns)
data_preped = data_proc.remove_backslash(data_preped, tcr_columns)
data_preped = data_proc.filter_clones_data(data_preped, tcr_columns)
data_preped = data_preped[-data_preped['reference.id'].str.startswith('https://www.10xgenomics',na=False)].reset_index(drop=True)
data_preped = data_preped[data_preped['species']=='HomoSapiens']
data_preped = data_proc.filter_segments(data_preped)
data_preped.shape

(36193, 17)

### frequent labels

In [12]:
data_preped = data_proc.freq_labels(label, data_id, data_preped, n = 15)
#data_preped = data_proc.freq_labels_tr_list(label, data_id, data_preped, [10,20,30,50,100,500,1000])
data_preped = data_proc.freq_labels(label_s, data_id, data_preped, n = 15)
#data_preped = data_proc.freq_labels_tr_list(label_s, data_id, data_preped, [10,20,30,50,100,500,1000])

### save

In [16]:
data_preped_train, data_preped_test = train_test_split(data_preped,test_size=0.3)

In [17]:
data_preped.to_csv(data_output_path, sep='\t',index=False)
data_preped_train.to_csv(data_train_output_path, sep='\t',index=False)
data_preped_test.to_csv(data_test_output_path, sep='\t',index=False)

In [18]:
list(data_preped[str(label + '_freq')].sort_values().drop_duplicates())

['CINGVCWTV',
 'ELAGIGILTV',
 'FRDYVDRFYKTLRAEQASQE',
 'GILGFVFTL',
 'GLCTLVAML',
 'KRWIILGLNK',
 'LLLGIGILV',
 'LLWNGPMAV',
 'NLVPMVATV',
 'PKYVKQNTLKLAT',
 'SPRWYFYYL',
 'TFEYVSQPFLMDLE',
 'TPRVTGGGAM',
 'TTDPSFLGRY',
 'YLQPRTFLL',
 'other']

In [ ]:
data_preped_test[data_preped_test[str(label + '_freq')]!='other']

,chain,cdr3aa,species,antigen.epitope,antigen.gene,antigen.species,complex.id,v,j,v.end,...,antigen.epitope_freq_500,antigen.epitope_freq_1000,antigen.species_freq,antigen.species_freq_10,antigen.species_freq_20,antigen.species_freq_30,antigen.species_freq_50,antigen.species_freq_100,antigen.species_freq_500,antigen.species_freq_1000
2051,TRB,CASSHGEGLAVSYNEQFF,HomoSapiens,LLLGIGILV,BST2,HomoSapiens,0,TRBV28,TRBJ2-1,4,...,LLLGIGILV,other,HomoSapiens,HomoSapiens,HomoSapiens,HomoSapiens,HomoSapiens,HomoSapiens,HomoSapiens,HomoSapiens
13761,TRB,CASSPRTSGGYQEPQYF,HomoSapiens,CINGVCWTV,NS3,HCV,1155,TRBV5-4,TRBJ2-5,4,...,CINGVCWTV,other,HCV,HCV,HCV,HCV,HCV,HCV,HCV,HCV
10038,TRA,CAESRTGNTPLVF,HomoSapiens,TTDPSFLGRY,NSP3,SARS-CoV-2,26612,TRAV13-2,TRAJ29,3,...,TTDPSFLGRY,other,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2
32785,TRB,CATSREGGGYNEQFF,HomoSapiens,PKYVKQNTLKLAT,HA,InfluenzaA,0,TRBV15,TRBJ2-1,5,...,PKYVKQNTLKLAT,other,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA
17130,TRA,CVLQPGRKAYL,HomoSapiens,GILGFVFTL,M,InfluenzaA,0,TRAV12-1,TRAJ23,2,...,GILGFVFTL,GILGFVFTL,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28313,TRA,CAGLWDRGSTLGRLYF,HomoSapiens,NLVPMVATV,pp65,CMV,0,TRAV35,TRAJ18,3,...,NLVPMVATV,NLVPMVATV,CMV,CMV,CMV,CMV,CMV,CMV,CMV,CMV
3422,TRB,CASSMRSTDTQYF,HomoSapiens,GILGFVFTL,M,InfluenzaA,"2697,1296,2693,7535,6759,23984,1229",TRBV19,TRBJ2-3,4,...,GILGFVFTL,GILGFVFTL,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA
17643,TRB,CASSYTTSGRSEQYF,HomoSapiens,LLWNGPMAV,NS4B,YFV,0,TRBV28,TRBJ2-7,4,...,LLWNGPMAV,other,YFV,YFV,YFV,YFV,YFV,YFV,YFV,other
35274,TRB,CSASGHHYGYTF,HomoSapiens,NLVPMVATV,pp65,CMV,0,TRBV20-1,TRBJ1-2,3,...,NLVPMVATV,NLVPMVATV,CMV,CMV,CMV,CMV,CMV,CMV,CMV,CMV


### subsets

In [ ]:
##Generate sunsets  by retaining epitopes having 10, 50, 100, 500, or 1000 cognate TCRs

In [25]:
samples_n = [10,50,100,500,1000]
freq_col_list = [f'{label}_freq_{i}' for i in samples_n]

In [14]:
data_preped_a = data_preped[data_preped['chain']=='TRA']
data_preped_b = data_preped[data_preped['chain']=='TRB']

In [15]:
label

'antigen.epitope'

In [16]:
data_preped_a = data_proc.freq_labels_tr_list(label, data_id, data_preped_a, samples_n)
data_preped_b = data_proc.freq_labels_tr_list(label, data_id, data_preped_b, samples_n)

In [28]:
chain = 'TRA'
dataset_outputs_suf = f'{outputs_path}VDJdb_data'
data_proc.filter_save_freq_subsets(data_preped_a, chain, label,samples_n, freq_col_list, dataset_outputs_suf)

In [29]:
chain = 'TRB'
dataset_outputs_suf = f'{outputs_path}VDJdb_data'
data_proc.filter_save_freq_subsets(data_preped_b, chain, label,samples_n, freq_col_list, dataset_outputs_suf)

### prep for atmtcr

In [22]:
atmtcr_trb_train_path = outputs_path + 'vdjdb_trb_atmtcr_train.csv'
atmtcr_trb_test_path = outputs_path + 'vdjdb_trb_atmtcr_test.csv'

In [40]:
data_preped_trb_train = data_preped_train[data_preped_train['chain']=='TRB'].reset_index(drop=True)
data_preped_trb_test = data_preped_test[data_preped_test['chain']=='TRB'].reset_index(drop=True)

In [41]:
data_preped_trb_train_atmtcr = data_preped_trb_train.rename({label:'Epitope','cdr3aa':'TCR'},axis=1)[['Epitope','TCR']]
data_preped_trb_test_atmtcr = data_preped_trb_test.rename({label:'Epitope','cdr3aa':'TCR'},axis=1)[['Epitope','TCR']]

In [42]:
data_preped_trb_train_atmtcr['Binding Affinity']=1
data_preped_trb_test_atmtcr['Binding Affinity']=1

In [43]:
import tcremb.ml_utils as ml_utils
data_preped_trb_train_atmtcr_n = ml_utils.generate_negative_pairs(data_preped_trb_train_atmtcr, len(data_preped_trb_test_atmtcr)*2 , 'TCR', 'Epitope')
data_preped_trb_train_atmtcr_n['Binding Affinity']=0
data_preped_trb_test_atmtcr_n = ml_utils.generate_negative_pairs(data_preped_trb_test_atmtcr, len(data_preped_trb_test_atmtcr)*2 , 'TCR', 'Epitope')
data_preped_trb_test_atmtcr_n['Binding Affinity']=0

In [44]:
data_preped_trb_train_atmtcr = pd.concat([data_preped_trb_train_atmtcr, data_preped_trb_train_atmtcr_n]).reset_index(drop=True)
data_preped_trb_test_atmtcr = pd.concat([data_preped_trb_test_atmtcr, data_preped_trb_test_atmtcr_n]).reset_index(drop=True)


In [48]:
data_preped_trb_test_atmtcr

,Epitope,TCR,Binding Affinity
0,NLVPMVATV,CASSFPLGGTYEQYF,1
1,NLVPMVATV,CASSLGLDTQYF,1
2,NLVPMVATV,CSARDLDRADNTGELFF,1
3,GTSGSPIVNR,CASLSGRAPQHF,1
4,NLVPMVATV,CASSLNYEQYF,1
...,...,...,...
21628,ELAGIGILTV,CASSPGEARGEQFF,0
21629,TTDPSFLGRY,CASSPGTLSTEAFF,0
21630,QYIKWPWYI,CASSSSFLGNQPQHF,0
21631,NLVPMVATV,CASSRFPTGSNNEQFF,0


In [18]:
#data_preped_trb_atmtcr_train, data_preped_trb_atmtcr_test = train_test_split(data_preped_trb_atmtcr,test_size=0.3)

In [50]:
data_preped_trb_train_atmtcr.to_csv(atmtcr_trb_train_path, index=False, header=False)
data_preped_trb_test_atmtcr.to_csv(atmtcr_trb_test_path, index=False, header=False)

## 10x donor 1

In [7]:
donor = 'donor1'
label = 'top_tetramer'

In [11]:
barcodes_path = '/home/ykremlyakova/projects/Archive/tcr_emb_mirpy_old/data/10x_' + donor + '/filtered_feature_bc_matrix/barcodes.tsv'
features_path = '/home/ykremlyakova/projects/Archive/tcr_emb_mirpy_old/data/10x_' + donor + '/filtered_feature_bc_matrix/features.tsv'
matrix_path = '/home/ykremlyakova/projects/Archive/tcr_emb_mirpy_old/data/10x_' + donor + '/filtered_feature_bc_matrix/matrix.mtx'
annot_path = '/home/ykremlyakova/projects/Archive/tcr_emb_mirpy_old/data/10x_' + donor + '/vdj_v1_hs_aggregated_' + donor + '_all_contig_annotations.csv'

In [12]:
matrix_path_preped = outputs_path + '10x_matrix_' + donor + '.txt'
data_output_path = outputs_path + '10x_annot_data_' + donor + '.txt'
data_paired_output_path = outputs_path + '10x_annot_data_paired' + donor + '.txt'

In [13]:
matrix_path_preped

'data/data_preped/10x_matrix_donor1.txt'

### 10x preprocessing - get matrix with values

### Get top tetramer

In [16]:
matrix = pd.read_csv(matrix_path_preped,sep='\t')
barcode_tetramer = data_proc.get_barcode_top_tetramer(matrix)
barcode_tetramer = barcode_tetramer[barcode_tetramer['top_tetramer']!='KLGGALQAK']
barcode_tetramer = barcode_tetramer[barcode_tetramer['count']>4]

### prep annot

In [17]:
data_annot = pd.read_csv(annot_path,sep=',')

In [22]:
data = pd.merge(data_annot, barcode_tetramer, on = pairing_id)

In [23]:
data = data.rename({'cdr3':'cdr3aa','v_gene':'v','j_gene':'j'},axis=1)

In [24]:
data = data_proc.annot_id(data, data_id)

In [25]:
data_preped = data_proc.remove_asterisk(data, tcr_columns)
data_preped = data_proc.remove_backslash(data_preped, tcr_columns)
data_preped = data_proc.filter_clones_data(data_preped, tcr_columns)
data_preped = data_preped[data_preped['high_confidence']==True]
data_preped = data_preped[data_preped['is_cell']==True]
data_preped = data_preped[data_preped['chain']!='Multi']

data_preped = data_proc.filter_segments(data_preped)
data_preped.shape

(56823, 21)
(36654, 21)


(27651, 21)

### Only 1 barcoce

In [26]:
data_preped = data_preped.sort_values(['umis','reads'],ascending=False).drop_duplicates(['barcode','chain'])
print(data_preped.shape)

(25673, 21)


### frequent labels

In [32]:
label_high = str(label + '_freq')

In [28]:
data_preped = data_proc.freq_labels(label, data_id, data_preped, n = 8)
data_preped = data_proc.freq_labels_tr_list(label, data_id, data_preped, [10,20,30,50,100,500,1000])

### save 

In [29]:
data_preped.to_csv(data_output_path,sep='\t', index = False)

In [31]:
data_preped

,barcode,is_cell,contig_id,high_confidence,length,chain,v,d_gene,j,c_gene,...,count,data_id,top_tetramer_freq,top_tetramer_freq_10,top_tetramer_freq_20,top_tetramer_freq_30,top_tetramer_freq_50,top_tetramer_freq_100,top_tetramer_freq_500,top_tetramer_freq_1000
13350,CTCTAATAGACTGTAA-23,True,CTCTAATAGACTGTAA-23_contig_1,True,701,TRB,TRBV7-6,TRBD2,TRBJ2-3,TRBC2,...,12,25812,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
12693,CTAGTGACACGTAAGG-1,True,CTAGTGACACGTAAGG-1_contig_1,True,734,TRB,TRBV7-2,TRBD1,TRBJ2-7,TRBC2,...,5,24499,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
23406,TCAGGATCAATGAAAC-2,True,TCAGGATCAATGAAAC-2_contig_1,True,672,TRB,TRBV4-1,TRBD1,TRBJ2-3,TRBC2,...,8,44988,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
20100,GTATTCTTCCTTTCTC-8,True,GTATTCTTCCTTTCTC-8_contig_1,True,699,TRB,TRBV27,NaN,TRBJ2-7,TRBC2,...,5,38742,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK
24199,TCTATTGCACAGACTT-11,True,TCTATTGCACAGACTT-11_contig_1,True,903,TRB,TRBV4-1,TRBD1,TRBJ2-3,TRBC2,...,7,46508,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2735,ACTGAACCAACGATGG-26,True,ACTGAACCAACGATGG-26_contig_2,True,484,TRA,TRAV35,NaN,TRAJ49,TRAC,...,141,5307,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK
11065,CGCTTCATCCTTGACC-24,True,CGCTTCATCCTTGACC-24_contig_2,True,501,TRA,TRAV35,NaN,TRAJ49,TRAC,...,7,21378,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
21890,TACCTATCACGACGAA-40,True,TACCTATCACGACGAA-40_contig_2,True,605,TRA,TRAV14DV4,NaN,TRAJ45,TRAC,...,15,42114,other,FLRGRAYGL,FLRGRAYGL,FLRGRAYGL,FLRGRAYGL,other,other,other
23774,TCGAGGCGTCTCTTTA-30,True,TCGAGGCGTCTCTTTA-30_contig_4,True,505,TRA,TRAV35,NaN,TRAJ49,TRAC,...,242,45709,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK


### Map pairs

In [33]:
data_a = data_preped[data_preped['chain']=='TRA'][['cdr3aa','v','j']+[ pairing_id, label, label_high]]
data_a = data_a.rename({'cdr3aa':'a_cdr3aa','v':'TRAV','j':'TRAJ'},axis=1)
data_b = data_preped[data_preped['chain']=='TRB'][['cdr3aa','v','j'] + [pairing_id, label, label_high]]
data_b = data_b.rename({'cdr3aa':'b_cdr3aa','v':'TRBV','j':'TRBJ'},axis=1)


In [34]:
data_paired = data_a.merge(data_b)
data_paired

,a_cdr3aa,TRAV,TRAJ,barcode,top_tetramer,top_tetramer_freq,b_cdr3aa,TRBV,TRBJ
0,CAVTDSWGKLQF,TRAV21,TRAJ24,GCAAACTAGAAGCCCA-26,AVFDRKSDAK,AVFDRKSDAK,CASKPTSTDTQYF,TRBV27,TRBJ2-3
1,CAYRSAKDSSYKLIF,TRAV38-2DV8,TRAJ12,AGAGCTTCAAGCCGTC-8,AVFDRKSDAK,AVFDRKSDAK,CASSSHPLATFTGELFF,TRBV7-2,TRBJ2-2
2,CAVDPMDTGRRALTF,TRAV8-3,TRAJ5,GTCATTTTCACCTCGT-27,AVFDRKSDAK,AVFDRKSDAK,CASSLGFTDTQYF,TRBV27,TRBJ2-3
3,CAVHGYGQNFVF,TRAV21,TRAJ26,CGACTTCCAAACCCAT-23,AVFDRKSDAK,AVFDRKSDAK,CASSQEWLAVSTDTQYF,TRBV4-3,TRBJ2-3
4,CATGDNYGQNFVF,TRAV17,TRAJ26,ACGGGCTAGGAGTACC-28,AVFDRKSDAK,AVFDRKSDAK,CASSDGQGLEQYF,TRBV27,TRBJ2-7
...,...,...,...,...,...,...,...,...,...
11695,CAGHTGNQFYF,TRAV35,TRAJ49,CGCTATCCAGTCGATT-4,IVTDFSVIK,IVTDFSVIK,CASSWGGGSHYGYTF,TRBV11-2,TRBJ1-2
11696,CAGHTGNQFYF,TRAV35,TRAJ49,ACTGAACCAACGATGG-26,IVTDFSVIK,IVTDFSVIK,CASSWGGGSHYGYTF,TRBV11-2,TRBJ1-2
11697,CAGHTGNQFYF,TRAV35,TRAJ49,CGCTTCATCCTTGACC-24,AVFDRKSDAK,AVFDRKSDAK,CSAQWTPGGNQPQHF,TRBV20-1,TRBJ1-5
11698,CAMREGVSGGGADGLTF,TRAV14DV4,TRAJ45,TACCTATCACGACGAA-40,FLRGRAYGL,other,CASSPPEREITDTQYF,TRBV18,TRBJ2-3


### save pairs

In [35]:
data_paired.to_csv(data_paired_output_path,sep='\t', index = False)

## VDJdb as train and 10x as pred

In [7]:
label = 'antigen.epitope'
label_high = 'antigen.epitope_freq'
data_train_path = outputs_path +'VDJdb_data.csv'
label_train = 'antigen.epitope'
label_train_high = 'antigen.epitope_freq'

donor = 'donor1'
data_pred_path = outputs_path + '10x_annot_data_' + donor + '.txt'
label_pred = 'top_tetramer'
label_pred_high = 'top_tetramer_freq'

data_output_path = outputs_path +'VDJdb_10x_train_pred.csv'

In [8]:
data_output_path

'data/data_preped/VDJdb_10x_train_pred.csv'

In [9]:
data_train = pd.read_csv(data_train_path,sep='\t')
data_pred = pd.read_csv(data_pred_path,sep='\t')

In [10]:
data_train

,chain,cdr3aa,species,antigen.epitope,antigen.gene,antigen.species,complex.id,v,j,v.end,...,antigen.epitope_freq_500,antigen.epitope_freq_1000,antigen.species_freq,antigen.species_freq_10,antigen.species_freq_20,antigen.species_freq_30,antigen.species_freq_50,antigen.species_freq_100,antigen.species_freq_500,antigen.species_freq_1000
0,TRA,CASNTGTASKLTF,HomoSapiens,GILGFVFTL,M,InfluenzaA,0,TRAV24,TRAJ44,2,...,GILGFVFTL,GILGFVFTL,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA
1,TRA,CAVLLEYGNKLVF,HomoSapiens,GILGFVFTL,M,InfluenzaA,0,TRAV12-2,TRAJ47,3,...,GILGFVFTL,GILGFVFTL,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA
2,TRB,CASSAFPCREGRNNEQFF,HomoSapiens,NLVPMVATV,pp65,CMV,0,TRBV6-1,TRBJ2-1,4,...,NLVPMVATV,NLVPMVATV,CMV,CMV,CMV,CMV,CMV,CMV,CMV,CMV
3,TRA,CVVSAIMDSNYQLIW,HomoSapiens,KTFPPTEPK,Nucleocapsid,SARS-CoV-2,29700,TRAV8-2,TRAJ33,4,...,other,other,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2
4,TRB,CASSHGVGQGPYEQYF,HomoSapiens,YLQPRTFLL,Spike,SARS-CoV-2,0,TRBV4-1,TRBJ2-7,4,...,YLQPRTFLL,YLQPRTFLL,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36188,TRB,CASSLKDSGSYEQYF,HomoSapiens,HTQGYFPD,Nef,HIV,0,TRBV4-3,TRBJ2-7,4,...,other,other,HIV,HIV,HIV,HIV,HIV,HIV,other,other
36189,TRA,CAMRELSNTGKLIF,HomoSapiens,NSAIGKIQY,Spike,SARS-CoV-2,29257,TRAV14DV4,TRAJ37,5,...,other,other,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2,SARS-CoV-2
36190,TRA,CALSEASGYSTLTF,HomoSapiens,NLVPMVATV,pp65,CMV,0,TRAV19,TRAJ11,5,...,NLVPMVATV,NLVPMVATV,CMV,CMV,CMV,CMV,CMV,CMV,CMV,CMV
36191,TRB,CASSQPQGPTDTQYF,HomoSapiens,GLIYNRMGAVTTEV,M1,InfluenzaA,0,TRBV6-6,TRBJ2-3,4,...,other,other,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA,InfluenzaA


In [11]:
data_pred

,barcode,is_cell,contig_id,high_confidence,length,chain,v,d_gene,j,c_gene,...,count,data_id,top_tetramer_freq,top_tetramer_freq_10,top_tetramer_freq_20,top_tetramer_freq_30,top_tetramer_freq_50,top_tetramer_freq_100,top_tetramer_freq_500,top_tetramer_freq_1000
0,CTCTAATAGACTGTAA-23,True,CTCTAATAGACTGTAA-23_contig_1,True,701,TRB,TRBV7-6,TRBD2,TRBJ2-3,TRBC2,...,12,25812,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
1,CTAGTGACACGTAAGG-1,True,CTAGTGACACGTAAGG-1_contig_1,True,734,TRB,TRBV7-2,TRBD1,TRBJ2-7,TRBC2,...,5,24499,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
2,TCAGGATCAATGAAAC-2,True,TCAGGATCAATGAAAC-2_contig_1,True,672,TRB,TRBV4-1,TRBD1,TRBJ2-3,TRBC2,...,8,44988,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
3,GTATTCTTCCTTTCTC-8,True,GTATTCTTCCTTTCTC-8_contig_1,True,699,TRB,TRBV27,NaN,TRBJ2-7,TRBC2,...,5,38742,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK
4,TCTATTGCACAGACTT-11,True,TCTATTGCACAGACTT-11_contig_1,True,903,TRB,TRBV4-1,TRBD1,TRBJ2-3,TRBC2,...,7,46508,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25668,ACTGAACCAACGATGG-26,True,ACTGAACCAACGATGG-26_contig_2,True,484,TRA,TRAV35,NaN,TRAJ49,TRAC,...,141,5307,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK
25669,CGCTTCATCCTTGACC-24,True,CGCTTCATCCTTGACC-24_contig_2,True,501,TRA,TRAV35,NaN,TRAJ49,TRAC,...,7,21378,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
25670,TACCTATCACGACGAA-40,True,TACCTATCACGACGAA-40_contig_2,True,605,TRA,TRAV14DV4,NaN,TRAJ45,TRAC,...,15,42114,other,FLRGRAYGL,FLRGRAYGL,FLRGRAYGL,FLRGRAYGL,other,other,other
25671,TCGAGGCGTCTCTTTA-30,True,TCGAGGCGTCTCTTTA-30_contig_4,True,505,TRA,TRAV35,NaN,TRAJ49,TRAC,...,242,45709,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK


In [12]:
data_train['data_type']='train'
data_pred['data_type']='pred'
data_train[label]=data_train[label_train]
data_train[label_high]=data_train[label_train_high]
data_pred[label]=data_pred[label_pred]
data_pred[label_high]=data_pred[label_pred_high]

In [13]:
data_tt = pd.concat([data_train,data_pred]).reset_index(drop=True)
data_tt

,chain,cdr3aa,species,antigen.epitope,antigen.gene,antigen.species,complex.id,v,j,v.end,...,top_tetramer,count,top_tetramer_freq,top_tetramer_freq_10,top_tetramer_freq_20,top_tetramer_freq_30,top_tetramer_freq_50,top_tetramer_freq_100,top_tetramer_freq_500,top_tetramer_freq_1000
0,TRA,CASNTGTASKLTF,HomoSapiens,GILGFVFTL,M,InfluenzaA,0,TRAV24,TRAJ44,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRA,CAVLLEYGNKLVF,HomoSapiens,GILGFVFTL,M,InfluenzaA,0,TRAV12-2,TRAJ47,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRB,CASSAFPCREGRNNEQFF,HomoSapiens,NLVPMVATV,pp65,CMV,0,TRBV6-1,TRBJ2-1,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRA,CVVSAIMDSNYQLIW,HomoSapiens,KTFPPTEPK,Nucleocapsid,SARS-CoV-2,29700,TRAV8-2,TRAJ33,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRB,CASSHGVGQGPYEQYF,HomoSapiens,YLQPRTFLL,Spike,SARS-CoV-2,0,TRBV4-1,TRBJ2-7,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61861,TRA,CAGHTGNQFYF,NaN,IVTDFSVIK,NaN,NaN,NaN,TRAV35,TRAJ49,NaN,...,IVTDFSVIK,141.0,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK
61862,TRA,CAGHTGNQFYF,NaN,AVFDRKSDAK,NaN,NaN,NaN,TRAV35,TRAJ49,NaN,...,AVFDRKSDAK,7.0,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK,AVFDRKSDAK
61863,TRA,CAMREGVSGGGADGLTF,NaN,FLRGRAYGL,NaN,NaN,NaN,TRAV14DV4,TRAJ45,NaN,...,FLRGRAYGL,15.0,other,FLRGRAYGL,FLRGRAYGL,FLRGRAYGL,FLRGRAYGL,other,other,other
61864,TRA,CAGHTGNQFYF,NaN,IVTDFSVIK,NaN,NaN,NaN,TRAV35,TRAJ49,NaN,...,IVTDFSVIK,242.0,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK,IVTDFSVIK


In [14]:
data_tt.to_csv(data_output_path, sep='\t',index=False)

In [15]:
data_output_path

'data/data_preped/VDJdb_10x_train_pred.csv'